In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from tqdm.notebook import tqdm
from typing import Optional, Callable, List

from PIL import Image

In [2]:
sys.path.append('../dataset')
sys.path.append('../graph')

In [3]:
from text_image_graph_dataset import TextImageGraphDataset
from torchvision.transforms import Compose, Normalize, Resize, InterpolationMode
from image_graph_dataset import ImageGraphDataset, rescale

In [6]:
text_root = '/deep/group/img-graph/radgraph_tokens'
image_root = '/deep/group/mimic-cxr-small/original'
graph_root = '/deep/group/img-graph'

input_resolution = 320
transform = Compose([
    Normalize((101.48761, 101.48761, 101.48761), (83.43944, 83.43944, 83.43944)),
])
    
dset = TextImageGraphDataset(
        name='train',
        text_root=text_root,
        image_root=image_root,
        graph_root=graph_root,
        image_pre_transform=rescale,
        image_transform=transform,
    )
dset

/deep/u/danieljm/miniconda3/envs/img-graph/lib/python3.8/site-packages/torch_geometric/data/dataset.py:159: UserWarning: The `pre_filter` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-fitering technique, make sure to delete '{self.processed_dir}' first
  warnings.warn(


TextImageGraphDataset(220736)

In [7]:
from transformers import CLIPTokenizer, CLIPTextModel
from torch_geometric.loader import DataLoader
import torch

from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'openai/clip-vit-base-patch32'
encoder = CLIPTextModel.from_pretrained(model_name).to(device)
tokenizer = CLIPTokenizer.from_pretrained(model_name)


for data in tqdm(dset):
    new_dir = '/'.join(data.pid.split('/')[:2])
    new_dir = os.path.join('/deep/group/img-graph/radgraph_mean_BERT_embeddings', new_dir)
    Path(new_dir).mkdir(parents=True, exist_ok=True)
    new_file = data.pid.split('/')[2][:-4] + '.pt'
    new_file = os.path.join(new_dir, new_file)
    
    if os.path.isfile(new_file):  # file already exists so continue
        continue
    
    inputs = tokenizer(data.tokens, padding=True, return_tensors="pt").to(device)
    # print('INPUTS DEVICE', inputs.device)
    outputs = encoder(**inputs)
    pooled_output = outputs.pooler_output
    avg_output = torch.mean(pooled_output, dim=0)  # average across tokens within a graph
    # print(data.pid, avg_output.shape)
    torch.save(avg_output, new_file)
    # break

You are using a model of type clip to instantiate a model of type clip_text_model. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPTextModel: ['vision_model.pre_layrnorm.bias', 'vision_model.encoder.layers.5.mlp.fc1.weight', 'vision_model.encoder.layers.1.self_attn.v_proj.bias', 'vision_model.encoder.layers.1.layer_norm1.weight', 'vision_model.encoder.layers.3.self_attn.k_proj.weight', 'vision_model.encoder.layers.4.mlp.fc1.weight', 'vision_model.encoder.layers.2.self_attn.q_proj.bias', 'vision_model.encoder.layers.5.mlp.fc1.bias', 'vision_model.encoder.layers.3.layer_norm1.weight', 'vision_model.encoder.layers.4.self_attn.q_proj.bias', 'vision_model.encoder.layers.9.layer_norm1.bias', 'vision_model.encoder.layers.8.layer_norm1.weight', 'vision_model.encoder.layers.3.layer_norm1.bias', 'vision_model.encoder.layers.8.layer_norm1.bias', 'vision_model.encod

  0%|          | 0/220736 [00:00<?, ?it/s]

ERROR: len(tokens) = 32 != 31 = graph.x.shape[0]
ERROR: len(tokens) = 42 != 41 = graph.x.shape[0]
ERROR: len(tokens) = 25 != 21 = graph.x.shape[0]
ERROR: len(tokens) = 30 != 28 = graph.x.shape[0]
ERROR: len(tokens) = 38 != 39 = graph.x.shape[0]
ERROR: len(tokens) = 26 != 24 = graph.x.shape[0]
ERROR: len(tokens) = 21 != 20 = graph.x.shape[0]
ERROR: len(tokens) = 28 != 27 = graph.x.shape[0]
ERROR: len(tokens) = 57 != 54 = graph.x.shape[0]
ERROR: len(tokens) = 57 != 54 = graph.x.shape[0]
ERROR: len(tokens) = 53 != 51 = graph.x.shape[0]
ERROR: len(tokens) = 28 != 27 = graph.x.shape[0]
ERROR: len(tokens) = 45 != 44 = graph.x.shape[0]
ERROR: len(tokens) = 20 != 19 = graph.x.shape[0]
ERROR: len(tokens) = 19 != 18 = graph.x.shape[0]
ERROR: len(tokens) = 36 != 35 = graph.x.shape[0]
ERROR: len(tokens) = 71 != 69 = graph.x.shape[0]
ERROR: len(tokens) = 57 != 55 = graph.x.shape[0]
ERROR: len(tokens) = 28 != 26 = graph.x.shape[0]
ERROR: len(tokens) = 30 != 29 = graph.x.shape[0]
ERROR: len(tokens) =